<a href="https://colab.research.google.com/github/jyothsna1076/AnamolyDetectionPRMLProject/blob/main/RandomForestPRML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.model_selection import train_test_split
np.random.seed(42)
df= pd.read_csv("Train_data.csv")
def le(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])

le(df)
x=  df.drop("class",axis= 1)
y= df["class"]
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= 0.5, random_state= 42)


In [32]:
from collections import Counter

class MyRandomForest:
    def __init__(self, n_trees=10, sample_ratio=0.3, max_depth=4, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.sample_ratio = sample_ratio
        self.n_features = n_features  # Number of random features to select for each tree
        self.trees = []
        self.feature_indices = []  # Store which features each tree used

    def bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, int(n_samples * self.sample_ratio), replace=True)
        return X.iloc[indices], y.iloc[indices]

    def fit(self, X, y):
        self.trees = []
        self.feature_indices = []

        n_total_features = X.shape[1]
        for _ in range(self.n_trees):
            # Select random subset of features
            if self.n_features is None:
                selected_features = np.arange(n_total_features)  # use all features
            else:
                selected_features = np.random.choice(n_total_features, self.n_features, replace=False)
            self.feature_indices.append(selected_features)

            X_sample, y_sample = self.bootstrap_sample(X.iloc[:, selected_features], y)

            tree = DecisionTreeClassifier(
                max_depth=self.max_depth
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = []
        for tree, features in zip(self.trees, self.feature_indices):
            preds = tree.predict(X.iloc[:, features])
            tree_predictions.append(preds)

        tree_predictions = np.array(tree_predictions).T
        final_predictions = [Counter(row).most_common(1)[0][0] for row in tree_predictions]
        return np.array(final_predictions)

In [39]:
import joblib
# best_sample, best_trees, best_depth = grid_search_rf(x, y, sample_ratios, n_trees_options, max_depths)
best_sample=0.5
best_trees= 15
best_depth = None
num_features = int(x_train.shape[1] * 0.3)
start_time = time.time()
final_model = MyRandomForest(n_trees=best_trees, sample_ratio=best_sample, max_depth=best_depth,n_features= num_features)
final_model.fit(x_train, y_train)
training_time = time.time() - start_time


joblib.dump(final_model, 'random_forset.pkl')
# print(f"Final Model Accuracy: {final_acc:.4f}")


['random_forset.pkl']

In [38]:
model =  joblib.load('random_forset.pkl')
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

# Print results
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
model =  joblib.load('random_forset.pkl')
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

# Print results
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Classification Report :\n")
print(classification_report(y_test, y_pred, target_names= ['Normal ', 'Anamoly']))

Accuracy:  0.9953
Precision: 0.9953
Recall:    0.9953
Accuracy:  0.9953
Precision: 0.9953
Recall:    0.9953
F1 Score:  0.9953
Training Time: 0.46 seconds
Classification Report :

              precision    recall  f1-score   support

     Normal        1.00      0.99      0.99      5878
     Anamoly       0.99      1.00      1.00      6718

    accuracy                           1.00     12596
   macro avg       1.00      1.00      1.00     12596
weighted avg       1.00      1.00      1.00     12596

